In [8]:
import skrf as rf
import numpy as np
from skrf.media import DistributedCircuit,Coaxial
import matplotlib.pyplot as plt


Q2.1 Define a transmission line with following per-unit-length parameters: L = 0.5 μH/m, C = 200 pF/m,
R = 4.0 Ohm/m, and G = 0.02 S/m, and calculate the propagation constant \gamma and characteristic impedance Z_0 at 800 MHz. If the line is 30 cm long, what is the attenuation in dB?

In [9]:
frequency1 = rf.Frequency(0.8,0.8,1,unit='ghz') #define frequency range
tl1 = DistributedCircuit(frequency=frequency1, z0=50, C=200e-12, L=0.5e-06, R=4.0, G=0.02)
line1 = tl1.line(30e-2,'m') # 30 cm length line 

print(f'propagation constant gamma {np.round(TL.gamma,2)},\n\
      Characteristic impedance{np.round(TL.Z0,2)} [Ohm],\n\
      loss at 800 MHz is {np.round(line.s21.s_db[:,0,0],2)} [dB]')

propagation constant gamma [0.54+50.27j],
      Characteristic impedance[49.99+0.46j] [Ohm],
      loss at 800 MHz is [-1.41] [dB]


Q2.3 RG-402U semirigid coaxial cable has an inner conductor diameter of 0.91 mm and a dielectric diameter of 3.02 mm. Both conductors are copper, and
the dielectric material is Teflon. Compute the R, L, G, and C parameters of this line at 1 GHz, and
use these results to find the characteristic impedance and attenuation of the line at 1 GHz. For reference, the manufacturer’s specifications of 50 Ohm is 0.43 dB/m

In [10]:
frequency2 = rf.Frequency(1,1,1) #1 point at GHz
coax1 = Coaxial(frequency=frequency2, z0=None,Dint=0.91e-3, Dout=3.02e-3, epsilon_r=2.1, sigma=5.8e7) #tanD = 0
print(f'Characteristic impedance Z0 = {coax1.Z0} [Ohm] \n\
Attenuation{10*np.log(np.exp(-coax1.alpha))} [dB/m]')

Characteristic impedance Z0 = [49.63249219-0.06182456j] [Ohm] 
Attenuation[-0.37832421] [dB/m]
